# Analysis of the Warsaw Stock Exchange Companies #

In [1]:
# Import necessary libraries

import os
import requests
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from bs4 import BeautifulSoup

In [2]:
# Set responses to get all companies and their ratios like book value per share, earnings per share, earnings before deducting interest and taxes, debt and others from Warsaw Stock Exchange. 

session = requests.Session()
response_pbv = session.get("https://www.biznesradar.pl/spolki-wskazniki-wartosci-rynkowej/akcje_gpw")
response_pbvgrah = session.get("https://www.biznesradar.pl/spolki-wskazniki-wartosci-rynkowej/akcje_gpw,1,CWKGrahamCurrent,2,2")
response_pe = session.get("https://www.biznesradar.pl/spolki-wskazniki-wartosci-rynkowej/akcje_gpw,1,CZCurrent,2,2")
response_dr = session.get("https://www.biznesradar.pl/spolki-wskazniki-zadluzenia/akcje_gpw")
response_pebit = session.get("https://www.biznesradar.pl/spolki-wskazniki-wartosci-rynkowej/akcje_gpw,1,CZOCurrent,2,2")
response_roe = session.get("https://www.biznesradar.pl/spolki-wskazniki-rentownosci/akcje_gpw")
response_income_net = session.get("https://www.biznesradar.pl/spolki-raporty-finansowe-rachunek-zyskow-i-strat/akcje_gpw")
response_income_ebit = session.get("https://www.biznesradar.pl/spolki-raporty-finansowe-rachunek-zyskow-i-strat/akcje_gpw,Q,IncomeEBIT,2,2,0")


In [3]:
# Let's define function to parsing table

def table_parser(response, table_number, number_of_columns=None):
    """Function take response argument and set html.parser. Next by defined table_number picks out table 
       where are alocated records and divides table for rows. In for loop divides every row for td tags 
       and strips them to row lists. Finally to avoid empty values in rows, checks by established argument 
       number_of_columns amount of values in list. Returns list of companies ratios."""
    
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find_all('table')[table_number]
    table_rows = table.find_all('tr')
    res = []
    for tr in table_rows:
        td = tr.find_all('td')
        row = [tr.text.strip() for tr in td if tr.text.strip()]
        if number_of_columns:
            if len(row) >= number_of_columns:
                res.append(row)
        else:
            if row:
                res.append(row)
    return res

# let's create Series

df = pd.DataFrame(table_parser(response_income_ebit, 0, 7))
df = df.drop(columns=[1, 2, 5, 6])
df = df.rename(columns={0: "Company", 3: "Income EBIT y/y [%]", 4: "Income EBIT q/q [%]"})
df_income_ebit = df.set_index("Company")


print(df_income_ebit.head())

df = pd.DataFrame(table_parser(response_income_net, 0, 7))
df = df.drop(columns=[1, 2, 5, 6])
df = df.rename(columns={0: "Company", 3: "Income net y/y [%]", 4: "Income net q/q [%]"})
df_income_net = df.set_index("Company")


print(df_income_net.head())

df = pd.DataFrame(table_parser(response_pe, 0, 4))
df = df.drop(columns=[1, 3, 4, 5, 6, 7, 8])
df = df.rename(columns={0: "Company", 2: "P/E"})
df_pe = df.set_index("Company")

print(df_pe.head())

df = pd.DataFrame(table_parser(response_pbvgrah, 0, 4))
df = df.drop(columns=[1, 3, 4, 5, 6, 7, 8])
df = df.rename(columns={0: "Company", 2: "P/BV Graham's"})
df_pbvgrah = df.set_index("Company")

print(df_pbvgrah.head())

df = pd.DataFrame(table_parser(response_pebit, 0, 4))
df = df.drop(columns=[1, 3, 4, 5, 6, 7, 8])
df = df.rename(columns={0: "Company", 2: "P/EBIT"})
df_pebit = df.set_index("Company")

print(df_pebit.head())

df = pd.DataFrame(table_parser(response_pbv, 0, 4))
df = df.drop(columns=[1, 3, 4, 5, 6, 7, 8])
df = df.rename(columns={0: "Company", 2: "P/BV"})
df_pbv = df.set_index("Company")

print(df_pbv.head())

df = pd.DataFrame(table_parser(response_dr, 0, 4))
df = df.drop(columns=[1, 3, 4, 5, 6, 7, 8])
df = df.rename(columns={0: "Company", 2: "Debt ratio"})
df_dr = df.set_index("Company")

print(df_dr)

df = pd.DataFrame(table_parser(response_roe, 0, 8))
df = df.drop(columns=[1, 3, 4, 5, 6, 7, 8])
df = df.rename(columns={0: "Company", 2: "ROE [%]"})
df_roe = df.set_index("Company")
print(df_roe)

                Income EBIT y/y [%] Income EBIT q/q [%]
Company                                                
CPS (CYFRPLSAT)            +716.64%            +504.27%
PKN (PKNORLEN)             +261.87%             +38.49%
MOL                        +181.73%             +29.53%
PZU                         +35.02%             +12.30%
OPL (ORANGEPL)            +1137.33%            +728.57%
               Income net y/y [%] Income net q/q [%]
Company                                             
PKN (PKNORLEN)            +52.36%            +23.86%
MOL                       +63.44%            +15.85%
PGN (PGNIG)               +95.70%            +19.91%
PGE                        +6.03%             +9.33%
PZU                       +15.23%            +11.37%
                   P/E
Company               
MWT (MWTRADE)   888.35
EAT (AMREST)    295.21
MLS (MLSYSTEM)  251.52
ERB (ERBUD)     236.36
SEK (SEKO)      163.01
                 P/BV Graham's
Company                       
INV (INVISTA) 

In [4]:
# We will join tables

df = pd.concat([df_pbv, df_pe, df_dr, df_pbvgrah, df_pebit, df_roe, df_income_net, df_income_ebit], axis=1, join="inner")

# Let's take a look on data

df.head(10)

,P/BV,P/E,Debt ratio,P/BV Graham's,P/EBIT,ROE [%],Income net y/y [%],Income net q/q [%],Income EBIT y/y [%],Income EBIT q/q [%]
Company,,,,,,,,,,
STS (SATIS),39.32,0.04,0.89,77.07,0.04,111 021.22%,+10.10%,-29.97%,-34.01%,-100.07%
LVC (LIVECHAT),33.01,27.48,0.22,57.06,25.61,120.13%,+29.11%,+13.80%,+49.88%,+27.42%
IFI (IFIRMA),15.36,46.03,0.19,13.39,36.94,33.37%,+50.31%,+14.18%,+214.10%,+94.56%
CDR (CDPROJEKT),10.03,16.64,0.15,20.40,16.56,60.30%,+38.23%,-47.07%,-44.59%,-80.31%
CRJ (CREEPYJAR),8.77,19.45,0.05,9.68,19.60,45.09%,+85.67%,+7.81%,+860.44%,+13.19%
TEN (TSGAMES),7.71,17.35,0.31,82.45,15.09,44.44%,-17.47%,-10.27%,-53.07%,-19.38%
MBR (MOBRUK),7.68,12.20,0.25,22.34,9.49,62.98%,+31.94%,-5.55%,+20.49%,+10.60%
PSW (PGSSOFT),7.47,13.39,0.32,9.95,12.38,55.78%,+60.96%,+14.79%,+143.45%,+26.46%
PLW (PLAYWAY),7.22,22.52,0.09,11.81,21.56,32.08%,+92.29%,+24.50%,+106.55%,+42.38%


In [5]:
df.isnull().sum()

P/BV                   0
P/E                    0
Debt ratio             0
P/BV Graham's          0
P/EBIT                 0
ROE [%]                0
Income net y/y [%]     0
Income net q/q [%]     0
Income EBIT y/y [%]    0
Income EBIT q/q [%]    0
dtype: int64

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 154 entries, STS (SATIS) to AAT (ALTA)
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   P/BV                 154 non-null    object
 1   P/E                  154 non-null    object
 2   Debt ratio           154 non-null    object
 3   P/BV Graham's        154 non-null    object
 4   P/EBIT               154 non-null    object
 5   ROE [%]              154 non-null    object
 6   Income net y/y [%]   154 non-null    object
 7   Income net q/q [%]   154 non-null    object
 8   Income EBIT y/y [%]  154 non-null    object
 9   Income EBIT q/q [%]  154 non-null    object
dtypes: object(10)
memory usage: 13.2+ KB


In [7]:
# We want to change our data types from object to float so first have to get rid of space, '+', and '%'

df['P/E'] = df['P/E'].str.replace(' ','', regex=False)
df['P/BV'] = df['P/BV'].str.replace(' ','', regex=False)
df['Debt ratio'] = df['Debt ratio'].str.replace(' ','', regex=False)
df["P/BV Graham's"] = df["P/BV Graham's"].str.replace(' ','', regex=False)
df['P/EBIT'] = df['P/EBIT'].str.replace(' ','', regex=False)
df['ROE [%]'] = df['ROE [%]'].str.replace(r'[\s+%]', '', regex=True)

df['Income net y/y [%]'] = df['Income net y/y [%]'].str.replace(r'[\s+%+]', '', regex=True)
df['Income net q/q [%]'] = df['Income net q/q [%]'].str.replace(r'[\s+%+]', '', regex=True)

df['Income EBIT y/y [%]'] = df['Income EBIT y/y [%]'].str.replace(r'[\s+%+]', '', regex=True)
df['Income EBIT q/q [%]'] = df['Income EBIT q/q [%]'].str.replace(r'[\s+%+]', '', regex=True)

df = df.astype(float)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 154 entries, STS (SATIS) to AAT (ALTA)
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   P/BV                 154 non-null    float64
 1   P/E                  154 non-null    float64
 2   Debt ratio           154 non-null    float64
 3   P/BV Graham's        154 non-null    float64
 4   P/EBIT               154 non-null    float64
 5   ROE [%]              154 non-null    float64
 6   Income net y/y [%]   154 non-null    float64
 7   Income net q/q [%]   154 non-null    float64
 8   Income EBIT y/y [%]  154 non-null    float64
 9   Income EBIT q/q [%]  154 non-null    float64
dtypes: float64(10)
memory usage: 13.2+ KB


In [9]:
# Now We can specify ratios that We are interested

df = df.loc[df['P/E'] <= 10].sort_values(by='P/E', ascending=False)
df = df.loc[df['P/BV'] <= 1]
df = df.loc[df['Debt ratio'] <= 1]
df = df.loc[df["P/BV Graham's"] <= 3]
df = df.loc[df['P/EBIT'] <= 10]

df = df.loc[df['Income net y/y [%]'] > 0]
df = df.loc[df['Income net q/q [%]'] > 0]

df = df.loc[df['Income EBIT y/y [%]'] > 0]
df = df.loc[df['Income EBIT q/q [%]'] > 0]

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10 entries, MXC (MAXCOM) to MVP (MARVIPOL)
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   P/BV                 10 non-null     float64
 1   P/E                  10 non-null     float64
 2   Debt ratio           10 non-null     float64
 3   P/BV Graham's        10 non-null     float64
 4   P/EBIT               10 non-null     float64
 5   ROE [%]              10 non-null     float64
 6   Income net y/y [%]   10 non-null     float64
 7   Income net q/q [%]   10 non-null     float64
 8   Income EBIT y/y [%]  10 non-null     float64
 9   Income EBIT q/q [%]  10 non-null     float64
dtypes: float64(10)
memory usage: 880.0+ bytes


In [10]:
# Let's take a look on output
df.head(40)

,P/BV,P/E,Debt ratio,P/BV Graham's,P/EBIT,ROE [%],Income net y/y [%],Income net q/q [%],Income EBIT y/y [%],Income EBIT q/q [%]
Company,,,,,,,,,,
MXC (MAXCOM),0.49,9.76,0.39,0.57,5.28,5.06,6.61,16.31,34.04,103.73
WAS (WASKO),0.61,8.52,0.43,1.71,5.77,7.12,38.56,29.95,264.09,48.84
IMP (IMPERIO),0.95,6.67,0.00,0.95,5.94,14.23,221.69,167.00,1606.28,2072.67
ABE (ABPL),0.84,6.65,0.67,1.22,4.99,12.71,7.76,4.78,23.06,1.97
HEL (HELIO),0.87,5.92,0.47,1.50,4.54,14.68,26.60,5.97,142.14,99.02
NTT (NTTSYSTEM),0.48,5.15,0.49,0.76,3.60,9.29,32.86,24.60,53.82,43.10
INP (INPRO),0.84,4.50,0.46,1.27,3.12,18.66,53.46,29.47,124.40,73.15
FSG (FASING),0.28,4.29,0.42,0.65,2.73,6.54,49.67,13.43,778.46,18.96
STP (STALPROD),0.50,3.72,0.24,1.23,3.13,13.51,50.75,7.36,181.45,22.79


In [28]:
# Now we can find out more information about one of the company above. We will set response to further analysis.

company = "ABE"
response_company = session.get(f"https://www.biznesradar.pl/raporty-finansowe-rachunek-zyskow-i-strat/{company}")
soup = BeautifulSoup(response_company.text, 'html.parser')

In [29]:
# First let's take a look how has been shaping dynamic of revenues over a years

years = soup.find(class_="report-table").find_all(class_=["thq h", "thq h newest"])
years = [record.text.strip() for record in years]
years = [record.replace("\n", "") for record in years]
years = [record.replace("\t", "") for record in years]

revenues = soup.find(class_="report-table").find_all("tr", {"data-field": "IncomeRevenues"})

for record in revenues: 
    values, changes = record.find_all(class_="value"), record.find_all(class_="changeyy")    
    
values = [value.text.strip() for value in values]
changes = [change.text.strip() for change in changes]
changes = [change.split(sep="~sektor") for change in changes]

dynamic = [change[0] for change in changes]
sector_dynamic = [change[1] for change in changes]

dynamic.insert(0, '0')
sector_dynamic.insert(0, '0')
revenues = zip(years, values, dynamic, sector_dynamic)

df = pd.DataFrame(revenues)

# We will change our data types from object to float so first have to replace unnecessary strings

df[1] = df[1].str.replace(r'\s+', '', regex=True)
df[2] = df[2].str.replace(r'[\s+r*\/%+]', '', regex=True)
df[3] = df[3].str.replace(r'[\s+%+]', '', regex=True)

df[1] = df[1].astype(float)
df[2] = df[2].astype(float)
df[3] = df[3].astype(float)

df = df.rename(columns={0: "Year", 1: "Revenue", 2: "Revenue y/y [%]", 3: "Sector revenue y/y [%]"})

# Let's plot diagram

fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Bar(x=df["Year"], y=df["Revenue"], name="revenues"),
    secondary_y=False,
)

fig.add_trace(
    go.Line(x=df["Year"], y=df["Revenue y/y [%]"], name="y/y dynamic [%]"),
    secondary_y=True,
)

fig.add_trace(
    go.Line(x=df["Year"], y=df["Sector revenue y/y [%]"], name="y/y sector dynamic [%]"),
    secondary_y=True,
)

fig.show()

In [30]:
# Here we will take insight to net and EBIT incomes

ebit = soup.find(class_="report-table").find_all("tr", {"data-field": "IncomeEBIT"})
income_net = soup.find(class_="report-table").find_all("tr", {"data-field": "IncomeNetProfit"})

for record in ebit: 
    ebit_values, ebit_changes = record.find_all(class_="value"), record.find_all(class_="changeyy")    
    
ebit_values = [value.text.strip() for value in ebit_values]
ebit_changes = [change.text.strip() for change in ebit_changes]
ebit_changes = [change.split(sep="~sektor") for change in ebit_changes]

ebit_dynamic = [change[0] for change in ebit_changes]
ebit_sector_dynamic = [change[1] for change in ebit_changes]

ebit_dynamic.insert(0, '0')
ebit_sector_dynamic.insert(0, '0')

for record in income_net: 
    income_net_values, income_net_changes = record.find_all(class_="value"), record.find_all(class_="changeyy")    
    
income_net_values = [value.text.strip() for value in income_net_values]
income_net_changes = [change.text.strip() for change in income_net_changes]
income_net_changes = [change.split(sep="~sektor") for change in income_net_changes]

income_net_dynamic = [change[0] for change in income_net_changes]
income_net_sector_dynamic = [change[1] for change in income_net_changes]

income_net_dynamic.insert(0, '0')
income_net_sector_dynamic.insert(0, '0')

incomes = zip(years, income_net_values, income_net_dynamic, income_net_sector_dynamic, ebit_values, ebit_dynamic, ebit_sector_dynamic)

df = pd.DataFrame(incomes)

# Let's replace unnecessary strings

df[1] = df[1].str.replace(r'\s+', '', regex=True)
df[2] = df[2].str.replace(r'[\s+r*\/%+]', '', regex=True)
df[3] = df[3].str.replace(r'[\s+%+]', '', regex=True)
df[4] = df[4].str.replace(r'\s+', '', regex=True)
df[5] = df[5].str.replace(r'[\s+r*\/%+]', '', regex=True)
df[6] = df[6].str.replace(r'[\s+%+]', '', regex=True)

# And change our data types from object to float

df[1] = df[1].astype(float)
df[2] = df[2].astype(float)
df[3] = df[3].astype(float)
df[4] = df[4].astype(float)
df[5] = df[5].astype(float)
df[6] = df[6].astype(float)

# Let's plot diagrams

fig = go.Figure()

fig.add_trace(go.Bar(x=df[0], y=df[1], name="net income"))
fig.add_trace(go.Bar(x=df[0], y=df[4], name="EBIT income"))

fig.update_layout(
    title='Net and EBIT income',
    xaxis_tickfont_size=14,
    yaxis=dict(
        title='PLN',
        titlefont_size=16,
        tickfont_size=14,
    ),
    legend=dict(
        x=0,
        y=1.0,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
    barmode='group',
    bargap=0.15,
    bargroupgap=0.1
)

fig2 = make_subplots(specs=[[{"secondary_y": True}]])

fig2.add_trace(
    go.Bar(x=df[0], y=df[1], name="net income"),
    secondary_y=False,
)

fig2.add_trace(
    go.Line(x=df[0], y=df[2], name="y/y income net dynamic [%]"),
    secondary_y=True,
)

fig2.add_trace(
    go.Line(x=df[0], y=df[3], name="y/y income net sector dynamic [%]"),
    secondary_y=True,
)

fig2.update_layout(
    title='Income net and dynamics',
    xaxis_tickfont_size=14,
    yaxis=dict(
        title='PLN',
        titlefont_size=16,
        tickfont_size=14,
    ),
    legend=dict(
        x=0,
        y=1.0,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
    barmode='group',
    bargap=0.15,
    bargroupgap=0.1
)

fig3 = make_subplots(specs=[[{"secondary_y": True}]])

fig3.add_trace(
    go.Bar(x=df[0], y=df[4], name="EBIT income"),
    secondary_y=False,
)

fig3.add_trace(
    go.Line(x=df[0], y=df[5], name="y/y income EBIT dynamic [%]"),
    secondary_y=True,
)

fig3.add_trace(
    go.Line(x=df[0], y=df[6], name="y/y income EBIT sector dynamic [%]"),
    secondary_y=True,
)

fig3.update_layout(
    title='EBIT income and dynamics',
    xaxis_tickfont_size=14,
    yaxis=dict(
        title='PLN',
        titlefont_size=16,
        tickfont_size=14,
    ),
    legend=dict(
        x=0,
        y=1.0,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
    barmode='group',
    bargap=0.15,
    bargroupgap=0.1
)

fig.show()
fig2.show()
fig3.show()

In [31]:
# Now let's take insight to return on equity

response_company_roe = session.get(f"https://www.biznesradar.pl/wskazniki-rentownosci/{company}")
soup = BeautifulSoup(response_company_roe.text, 'html.parser')

In [32]:
dates = soup.find(class_="report-table").find_all(class_=["thq h", "thq h newest"])
dates = [record.text.strip() for record in dates]
dates = [record.replace("\n", "") for record in dates]
dates = [record.replace("\t", "") for record in dates]
dates = [record[0:7] for record in dates]

roe = soup.find(class_="report-table").find_all("tr", {"data-field": "ROE"})

for record in roe: 
    values = record.find_all(class_="value")  
    
values = [value.text.strip() for value in values]

roe = zip(dates, values)

df = pd.DataFrame(roe)
df[1] = df[1].str.replace(r'[\s+%+]', '', regex=True)
df[1] = df[1].astype(float)

# Let's plot

fig = go.Figure()

fig.add_trace(go.Line(x=df[0], y=df[1], name="ROE"))

fig.update_layout(
    title='Return on equity [ROE]',
    xaxis_tickfont_size=14,
    yaxis=dict(
        title='[%]',
        titlefont_size=16,
        tickfont_size=14,
    ),
    legend=dict(
        x=0,
        y=1.0,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
    barmode='group',
    bargap=0.15,
    bargroupgap=0.1
)

fig.show() 

In [16]:
# Now let's find out about capitalization and book value

response_company = session.get(f"https://www.biznesradar.pl/wskazniki-wartosci-rynkowej/{company}")
soup = BeautifulSoup(response_company.text, 'html.parser')

th = soup.find(class_="report-table").find_all(class_="thq h newest")
tr = soup.find(class_="report-table").find_all("tr", {"data-field": "WK"})
td = soup.find(class_="report-table").find_all(class_="value")

# We get book value per stock from recent report 

def book_value_per_stock(data):
    values = []
    for record in tr:
        records = record.find_all(class_="value")
        for record in records:
            values.append(record.text.strip())
    return values[-1]

book_value_per_stock = book_value_per_stock(tr)
last_report_date = th[-1].text.strip()[0:7]

print(f"Book value per share: {book_value_per_stock} pln. Report quarter {last_report_date}.")

# Here we want to find out about company market capitalization

th = soup.find(class_="profileSummary").find_all('th')
td = soup.find(class_="profileSummary").find_all('td')


th_strip = [record.text.strip() for record in th]
td_strip = [record.text.strip() for record in td]

summary = zip(th_strip, td_strip)

df = pd.DataFrame(summary)

# Add book value per stock to DataFrame
df = df.append({0: f"BV per share {last_report_date}", 1: book_value_per_stock}, ignore_index=True)

# Replace unnecessary types and calculate book value by multiplying number of stocks and book value per stock
df[1] = df[1].str.replace(' ','')
book_value = float(book_value_per_stock) * float(df[1][2])
df = df.append({0: 'Book Value', 1: book_value}, ignore_index=True)
df = df.set_index(0)

df.head(10)

Book value per share: 9.17 pln. Report quarter 2021/Q3.


,1
0,
ISIN:,PLINPRO00015
Data debiutu:,22.03.2011
Liczba akcji:,40040000
Kapitalizacja:,304304000
Sektor:,GPW:Deweloperzy
Branża:,Nieruchomości
BV per share 2021/Q3,9.17
Book Value,367166800.0


In [17]:
# Here we download last news about company

#company = 'NTT'
response_news = session.get(f"https://www.biznesradar.pl/wiadomosci/{company}")
soup = BeautifulSoup(response_news.text, 'html.parser')


In [18]:
# Let's parse website to dates, header, body and links

record_header = soup.find_all(class_="record-header")
record_body = soup.find_all(class_="record-body")
record_date = soup.find_all(class_='record-date')

dates = [date.text.strip() for date in record_date]
headers = [header.text.strip() for header in record_header]
bodies = [body.text.strip() for body in record_body]
links = []

# Here we get into record_header to get links
for record in record_header:
    hrefs = record.find_all("a")
    for href in hrefs:
        links.append(href["href"])

# Now we zip header, body and links
news = zip(dates, headers, bodies, links)

# And create DataFrame
df = pd.DataFrame(news)
df = df.set_index(0)

# Add function which allow links to be clickable
def make_clickable(path):
    
    # returns the final component of a path
    f_url = os.path.basename(path)
      
    # convert the path into clickable link
    return '<a href="{}">{}</a>'.format(path, f_url)
  
# Applying function make_clickable on column 3
df = df.style.format({3: make_clickable})
df


,1,2,3
0,,,
2021-11-24 15:54:57,Grupa INPRO poprawia rentowność,"Grupa INP (INPRO) podsumowała wyniki finansowe za trzy kwartały 2021 roku. W tym okresie uzyskała skonsolidowane przychody netto ze sprzedaży na poziomie 194,1 mln zł oraz wypracowała 28,2...","105397,grupa-inpro-poprawia-rentownosc"
2021-11-24 07:30:46,INPRO SA Raport okresowy kwartalny skonsolidowany 3/2021 QSr,Raport okresowy kwartalny skonsolidowany 3/2021 QSr,"40927892,inpro-sa-raport-okresowy-kwartalny-skonsolidowany-3-2021-qsr"
2021-11-16 14:37:53,INPRO SA (31/2021) Informacja o odwołaniu Wiceprezesa Zarządu Spółki Pana Piotra Stefaniaka i zmianie ilości osobowej składu Zarządu Spółki.,Raport bieżący 31/2021,"40859961,inpro-sa-31-2021-informacja-o-odwolaniu-wiceprezesa-zarzadu-spolki-pana-piotra-stefaniaka-i-zmianie-ilosci-osobowej-skladu-zarzadu-spolki"
2021-11-15 12:12:46,INPRO SA (30/2021) Podpisanie umowy znaczącej z Alior Bank SA.,Raport bieżący 30/2021,"40847630,inpro-sa-30-2021-podpisanie-umowy-znaczacej-z-alior-bank-sa"
2021-10-22 14:18:32,"INPRO SA (29/2021) Dokonanie wpisu hipoteki na nieruchomości, której Inpro SA jest właścicielem.",Raport bieżący 29/2021,"40643929,inpro-sa-29-2021-dokonanie-wpisu-hipoteki-na-nieruchomosci-ktorej-inpro-sa-jest-wlascicielem"
2021-10-13 14:36:11,INPRO SA (28/2021) Wyznaczenie pierwszego dnia notowania obligacji w alternatywnym systemie obrotu na Catalyst.,Raport bieżący 28/2021,"40561602,inpro-sa-28-2021-wyznaczenie-pierwszego-dnia-notowania-obligacji-w-alternatywnym-systemie-obrotu-na-catalyst"
2021-10-07 13:58:43,INPRO SA (27/2021) Emisja obligacji serii C,Raport bieżący 27/2021,"40513950,inpro-sa-27-2021-emisja-obligacji-serii-c"
2021-10-06 15:59:34,Po trzech kwartałach 2021 r. Grupa INPRO zwiększyła sprzedaż o 22%,Grupa Kapitałowa INP (INPRO) podsumowała wyniki sprzedaży za III kwartał 2021 roku. Od czerwca do końca września bieżącego roku Grupa zrealizowała przedsprzedaż (rozumianą jako liczbę...,"103981,po-trzech-kwartalach-2021-r-grupa-inpro-zwiekszyla-sprzedaz-o-22"
2021-10-06 12:40:19,INPRO SA (26/2021) Informacja nt. liczby zawartych umów przedwstępnych sprzedaży lokali w III kwartale 2021 roku.,Raport bieżący 26/2021,"40503095,inpro-sa-26-2021-informacja-nt-liczby-zawartych-umow-przedwstepnych-sprzedazy-lokali-w-iii-kwartale-2021-roku"
